### Audio data in Pytorch Lightning - example using simple built-in dataset

In [ ]:
!pip install torchaudio
!pip install lightning

In [ ]:
import lightning as L
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchaudio
from torchaudio.datasets import SPEECHCOMMANDS
from torchaudio.transforms import MelSpectrogram, Resample
from lightning.pytorch.callbacks import EarlyStopping, TQDMProgressBar
import torchmetrics
import os
import pathlib

In [ ]:
# https://pytorch.org/audio/stable/generated/torchaudio.transforms.MelSpectrogram.html
# https://pytorch.org/audio/stable/tutorials/audio_feature_extractions_tutorial.html#sphx-glr-tutorials-audio-feature-extractions-tutorial-py

# https://en.wikipedia.org/wiki/Mel_scale

class SubsetSC(SPEECHCOMMANDS):
    def __init__(self, subset: str = None):
        super().__init__("./data", download=True)

        def load_list(filename):
            filepath = os.path.join(self._path, filename)
            with open(filepath) as fileobj:
                return [os.path.normpath(os.path.join(self._path, line.strip())) for line in fileobj]

        if subset == "validation":
            self._walker = load_list("validation_list.txt")
        elif subset == "testing":
            self._walker = load_list("testing_list.txt")
        elif subset == "training":
            excludes = set(load_list("validation_list.txt") + load_list("testing_list.txt"))
            self._walker = [w for w in self._walker if w not in excludes]

    def __getitem__(self, n):
        fileid = self._walker[n]
        waveform, sample_rate = torchaudio.load(fileid)

        # Extract label from the file path
        label = os.path.basename(os.path.dirname(fileid))

        return waveform, sample_rate, label

class AudioDataModule(L.LightningDataModule):
    def __init__(self, batch_size=32, data_dir="./data", sample_rate=16000):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.sample_rate = sample_rate
        self.transform = MelSpectrogram(
            sample_rate=sample_rate,
            n_fft=1024,
            hop_length=512,
            n_mels=64
        )

    def prepare_data(self):
        pathlib.Path(self.data_dir).mkdir(parents=True, exist_ok=True)
        SPEECHCOMMANDS(root=self.data_dir, download=True)

    def setup(self, stage=None):
        self.train_data = SubsetSC("training")
        self.val_data = SubsetSC("validation")

        # Get the list of commands (classes)
        self.labels = sorted([
            'backward', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'follow',
            'forward', 'four', 'go', 'happy', 'house', 'learn', 'left', 'marvin', 'nine',
            'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three',
            'tree', 'two', 'up', 'visual', 'wow', 'yes', 'zero'
        ])
        self.label_to_idx = {label: idx for idx, label in enumerate(self.labels)}

    def process_audio(self, waveform, sample_rate):
        if sample_rate != self.sample_rate:
            resampler = Resample(sample_rate, self.sample_rate)
            waveform = resampler(waveform)

        mel_spec = self.transform(waveform)
        return mel_spec.squeeze(0)

    def collate_batch(self, batch):
        waveforms = []
        labels = []

        # First, process all waveforms and find the maximum length
        processed_waves = []
        max_length = 0

        for waveform, sample_rate, command in batch:
            try:
                processed = self.process_audio(waveform, sample_rate)
                processed_waves.append(processed)
                max_length = max(max_length, processed.size(1))
                labels.append(self.label_to_idx[command])
            except KeyError as e:
                print(f"Warning: Unknown command {command}")
                continue

        if not processed_waves:
            raise RuntimeError("No valid samples in batch")

        # Pad all spectrograms to the same length
        padded_waves = []
        for wave in processed_waves:
            if wave.size(1) < max_length:
                padding = torch.zeros(64, max_length - wave.size(1))
                padded_wave = torch.cat([wave, padding], dim=1)
                padded_waves.append(padded_wave)
            else:
                padded_waves.append(wave)

        waveforms = torch.stack(padded_waves)
        labels = torch.tensor(labels)
        return waveforms, labels

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=self.collate_batch,
            num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            collate_fn=self.collate_batch,
            num_workers=2
        )


In [ ]:
class AudioClassifier(L.LightningModule):
    def __init__(self, n_classes, n_mels=64):
        super().__init__()

        self.save_hyperparameters()

        self.gru = nn.GRU(
            input_size=n_mels,
            hidden_size=256,
            num_layers=2,
            batch_first=True,
            dropout=0.3
        )

        self.fc = nn.Linear(256, n_classes)

        metrics = torchmetrics.MetricCollection([
            torchmetrics.Accuracy(task='multiclass', num_classes=n_classes),
            torchmetrics.F1Score(task='multiclass', num_classes=n_classes)
            # https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall
            # https://scikit-learn.org/1.5/modules/generated/sklearn.metrics.f1_score.html
            # https://deepgram.com/ai-glossary/f1-score-machine-learning
            # more than 0.9 -> very good; less than 0.5 - very bad

        ])

        self.train_metrics = metrics.clone(prefix='train_')
        self.val_metrics = metrics.clone(prefix='val_')

    def forward(self, x):
        x = x.transpose(1, 2)
        output, hidden = self.gru(x)
        return self.fc(hidden[-1])

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)

        metrics = self.train_metrics(logits, y)
        self.log('train_loss', loss, prog_bar=True)
        self.log_dict(metrics, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)

        metrics = self.val_metrics(logits, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log_dict(metrics, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=2,
            verbose=True
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss"
            }
        }

In [ ]:
class LitProgressBar(TQDMProgressBar):
    def get_metrics(self, trainer, model):
        items = super().get_metrics(trainer, model)
        items.pop("v_num", None)
        return items

In [ ]:
L.seed_everything(42)

data_module = AudioDataModule()
data_module.prepare_data()
data_module.setup()

print(f"Number of classes: {len(data_module.labels)}")
print("Available commands:", data_module.labels)

model = AudioClassifier(
    n_classes=len(data_module.labels)
)

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        mode='min'
    ),
    LitProgressBar(refresh_rate=10)
]

trainer = L.Trainer(
    max_epochs=10,
    accelerator='auto',
    callbacks=callbacks,
    val_check_interval=0.25,
    log_every_n_steps=10,
    enable_progress_bar=True,
    enable_model_summary=True
)

In [ ]:
trainer.fit(model, data_module)

### Task a:
Change GRU layer to something you know

### Task b:
Using the docs do the same thing with the torchtext library
Do it using the AmazonReviewPolarity dataset
https://pytorch.org/text/main/datasets.html#amazonreviewpolarity


In [ ]:
!pip install torchtext

In [ ]:
from torchtext.datasets import AmazonReviewFull
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator

### Simple hyperparameter tuning with Ray Tune

This is not the only way to tune the model, as there are multiple similiar tools (Optuna, Skorch etc).

Based on: https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html


In [ ]:
!pip install ray[tune]

In [ ]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
# https://docs.ray.io/en/latest/tune/api/doc/ray.tune.schedulers.AsyncHyperBandScheduler.html
import ray.cloudpickle as pickle

In [ ]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )

    return trainset, testset

In [ ]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train_cifar(config, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    checkpoint = get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "rb") as fp:
                checkpoint_state = pickle.load(fp)
            start_epoch = checkpoint_state["epoch"]
            net.load_state_dict(checkpoint_state["net_state_dict"])
            optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8
    )

    for epoch in range(start_epoch, 10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.inference_mode():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }
        with tempfile.TemporaryDirectory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "wb") as fp:
                pickle.dump(checkpoint_data, fp)

            checkpoint = Checkpoint.from_directory(checkpoint_dir)
            train.report(
                {"loss": val_loss / val_steps, "accuracy": correct / total},
                checkpoint=checkpoint,
            )

    print("Finished Training")

In [ ]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.inference_mode():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [ ]:
config = {
    "l1": tune.choice([2 ** i for i in range(9)]),
    "l2": tune.choice([2 ** i for i in range(9)]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [ ]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.choice([2**i for i in range(9)]),
        "l2": tune.choice([2**i for i in range(9)]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16]),
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="accuracy", mode="max")
    with best_checkpoint.as_directory() as checkpoint_dir:
        data_path = Path(checkpoint_dir) / "data.pkl"
        with open(data_path, "rb") as fp:
            best_checkpoint_data = pickle.load(fp)

        best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])
        test_acc = test_accuracy(best_trained_model, device)
        print("Best trial test set accuracy: {}".format(test_acc))


In [ ]:
main(num_samples=10, max_num_epochs=10, gpus_per_trial=1)

### Tuning using PyTorch Lightning
https://docs.ray.io/en/latest/tune/examples/tune-pytorch-lightning.html

In [ ]:
!pip install lightning
!pip install ray[tune]

In [ ]:
import os
import torch
import tempfile
import lightning.pytorch as pl
import torch.nn.functional as F
from filelock import FileLock
from torchmetrics import Accuracy
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms

In [ ]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self, config):
        super(MNISTClassifier, self).__init__()
        self.accuracy = Accuracy(task="multiclass", num_classes=10, top_k=1)
        self.layer_1_size = config["layer_1_size"]
        self.layer_2_size = config["layer_2_size"]
        self.lr = config["lr"]

        # mnist images are (1, 28, 28) (channels, width, height)
        self.layer_1 = torch.nn.Linear(28 * 28, self.layer_1_size)
        self.layer_2 = torch.nn.Linear(self.layer_1_size, self.layer_2_size)
        self.layer_3 = torch.nn.Linear(self.layer_2_size, 10)
        self.eval_loss = []
        self.eval_accuracy = []

    def cross_entropy_loss(self, logits, labels):
        return F.nll_loss(logits, labels)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = x.view(batch_size, -1)

        x = self.layer_1(x)
        x = torch.relu(x)

        x = self.layer_2(x)
        x = torch.relu(x)

        x = self.layer_3(x)
        x = torch.log_softmax(x, dim=1)

        return x

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        accuracy = self.accuracy(logits, y)

        self.log("ptl/train_loss", loss)
        self.log("ptl/train_accuracy", accuracy)
        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        accuracy = self.accuracy(logits, y)
        self.eval_loss.append(loss)
        self.eval_accuracy.append(accuracy)
        return {"val_loss": loss, "val_accuracy": accuracy}

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.eval_loss).mean()
        avg_acc = torch.stack(self.eval_accuracy).mean()
        self.log("ptl/val_loss", avg_loss, sync_dist=True)
        self.log("ptl/val_accuracy", avg_acc, sync_dist=True)
        self.eval_loss.clear()
        self.eval_accuracy.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer



In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=128):
        super().__init__()
        self.data_dir = tempfile.mkdtemp()
        self.batch_size = batch_size
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )

    def setup(self, stage=None):
        with FileLock(f"{self.data_dir}.lock"):
            mnist = MNIST(
                self.data_dir, train=True, download=True, transform=self.transform
            )
            self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])

            self.mnist_test = MNIST(
                self.data_dir, train=False, download=True, transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=4)

In [ ]:
default_config = {
    "layer_1_size": 128,
    "layer_2_size": 256,
    "lr": 1e-3,
}

In [ ]:
from ray.train.lightning import (
    RayDDPStrategy,
    RayLightningEnvironment,
    RayTrainReportCallback,
    prepare_trainer,
)


In [ ]:
def train_func(config):
    dm = MNISTDataModule(batch_size=config["batch_size"])
    model = MNISTClassifier(config)

    trainer = pl.Trainer(
        devices="auto",
        accelerator="auto",
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
        enable_progress_bar=False,
    )
    trainer = prepare_trainer(trainer)
    trainer.fit(model, datamodule=dm)

In [ ]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler, AsyncHyperBandScheduler
# https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/
# https://arxiv.org/abs/1810.05934
# https://docs.ray.io/en/latest/tune/api/doc/ray.tune.schedulers.AsyncHyperBandScheduler.html


In [ ]:
search_space = {
    "layer_1_size": tune.choice([32, 64, 128]),
    "layer_2_size": tune.choice([64, 128, 256]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([32, 64]),
}

In [ ]:
# The maximum training epochs
num_epochs = 3 # 5

# Number of sampls from parameter space
num_samples = 5 # 10

In [ ]:
from ray.train import RunConfig, ScalingConfig, CheckpointConfig

scaling_config = ScalingConfig(
    num_workers=1, use_gpu=True, resources_per_worker={"CPU": 1, "GPU": 1}
    # num_workers=2, use_gpu=True, resources_per_worker={"CPU": 1, "GPU": 1} # based on hardware configuration; if incorrect will not run (waiting for resources)
)

run_config = RunConfig(
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="ptl/val_accuracy",
        checkpoint_score_order="max",
    ),
)

In [ ]:
from ray.train.torch import TorchTrainer

# Define a TorchTrainer without hyper-parameters for Tuner
ray_trainer = TorchTrainer(
    train_func,
    scaling_config=scaling_config,
    run_config=run_config,
)

In [ ]:
# https://docs.ray.io/en/latest/tune/api/doc/ray.tune.schedulers.AsyncHyperBandScheduler.html

def tune_mnist_asha(num_samples=10):
    scheduler = ASHAScheduler(max_t=num_epochs, grace_period=1, reduction_factor=2)

    tuner = tune.Tuner(
        ray_trainer,
        param_space={"train_loop_config": search_space},
        tune_config=tune.TuneConfig(
            metric="ptl/val_accuracy",
            mode="max",
            num_samples=num_samples,
            scheduler=scheduler,
        ),
    )
    return tuner.fit()



In [ ]:
# https://docs.ray.io/en/latest/tune/api/suggestion.html#random-search-and-grid-search-tune-search-basic-variant-basicvariantgenerator

results = tune_mnist_asha(num_samples=num_samples)

### Optimizing using CometML
https://www.comet.com/docs/v2/guides/optimizer/quickstart/
https://www.comet.com/docs/v2/guides/quickstart/




In [ ]:
!pip install comet_ml

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
import comet_ml
from google.colab import userdata

# Initialize the Comet SDK

# log in with prompt for key
# comet_ml.login(project_name="DLF_lab7_hyper_autotune_quickstart")


# log in with file (pass path to directory with config file)
# set up ".comet.config" file with API key
# create a file with following contents:
'''
[comet]
api_key = <YOUR_API_KEY>
'''
# comet_ml.login(project_name="DLF_lab7_hyper_autotune_quickstart", directory="./")

# log in with secret from google colab
comet_ml.login(userdata.get('COMET_API_KEY'), project_name="DLF_lab7_hyper_autotune_quickstart")


# experiment = comet_ml.start(project_name="DLF_lab7_hyper_autotune_quickstart")
#you can rename the experiment so it will be easier to find
# experiment.set_name("very_clever_name")


In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical


def build_model_graph(experiment):
    model = Sequential()
    model.add(
        Dense(
            experiment.get_parameter("first_layer_units"),
            activation="sigmoid",
            input_shape=(784,),
        )
    )
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"],
    )
    return model


def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        x_train,
        y_train,
        batch_size=experiment.get_parameter("batch_size"),
        epochs=experiment.get_parameter("epochs"),
        validation_data=(x_test, y_test),
    )


def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Score %s", score)


def get_dataset():
    num_classes = 10

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype("float32")
    x_test = x_test.astype("float32")
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], "train samples")
    print(x_test.shape[0], "test samples")

    # convert class vectors to binary class matrices
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test



In [ ]:
# Get the dataset:
x_train, y_train, x_test, y_test = get_dataset()

In [ ]:
# The optimization config:
config = {
    "algorithm": "bayes",  # "random" ; "grid"
    "name": "Optimize MNIST Network - Keras implementation (MLP)",
    "spec": {"maxCombo": 5, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {
            "type": "integer",
            "mu": 500,
            "sigma": 50,
            "scalingType": "normal",
        },
        "batch_size": {"type": "discrete", "values": [64, 128, 256]},
    },
    "trials": 1,
}



'''
    "parameters": {
        "learning_rate": {"type": "float", "scaling_type": "log_uniform", "min": 0.00001, "max": 0.001},
        "batch_size": {"type": "discrete", "values": [32, 64, 128, 256]},
    },
'''

opt = comet_ml.Optimizer(config)

In [ ]:
for experiment in opt.get_experiments():
    # Log parameters, or others:
    experiment.log_parameter("epochs", 10)

    # Build the model:
    model = build_model_graph(experiment)

    # Train it:
    train(experiment, model, x_train, y_train, x_test, y_test)

    # How well did it do?
    evaluate(experiment, model, x_test, y_test)

    # Optionally, end the experiment:
    experiment.end()

### Task 1:

Based on previous code create an automatic hyperparameter tuning test, using PyTorch or PyTorch Lightning, the Imaginette dataset, chosen (modifiable) network architecture and visualize the process using CometML

In [5]:
import comet_ml
import einops
import torch.nn as nn

class ImaginetteCNN(nn.Module):

    @staticmethod
    def depthwise_separable_conv(in_channels, out_channels, kernel_size=3, stride=1, padding=1, dropout_p=0.2):
        return nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size, stride, padding, groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels),
        )

    def __init__(self, num_classes=10, **kwargs):
        super().__init__()

        self.features = nn.Sequential(
            # Entry flow
            nn.Conv2d(3, 32, kernel_size=3, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            ImaginetteCNN.depthwise_separable_conv(64, 128),
            nn.ReLU(),           
            ImaginetteCNN.depthwise_separable_conv(128, 128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            ImaginetteCNN.depthwise_separable_conv(128, 256),
            nn.ReLU(),           
            ImaginetteCNN.depthwise_separable_conv(256, 256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),

            # Middle flow
            ImaginetteCNN.depthwise_separable_conv(256, 256),
            nn.ReLU(),           
            ImaginetteCNN.depthwise_separable_conv(256, 256),
            nn.ReLU(),
            ImaginetteCNN.depthwise_separable_conv(256, 256),
            nn.ReLU(),

            # Exit flow
            ImaginetteCNN.depthwise_separable_conv(256, 512),
            nn.ReLU(),
            ImaginetteCNN.depthwise_separable_conv(512, 1024),
            nn.ReLU(),

            nn.AdaptiveAvgPool2d((1,1))
        )

        self.classifier = nn.Sequential(
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(in_features=2048, out_features=num_classes)
        )

    def forward(self, x): 
        x = self.features(x)
        x = einops.rearrange(x, "b c w h -> b (c w h)")
        x = self.classifier(x)
        return x

In [ ]:
from config import API_KEY
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as v2
import torchvision.datasets as datasets
import torchvision.models as models
from ray import tune
from ray.air.integrations.comet import CometLoggerCallback
from ray.tune.schedulers import ASHAScheduler
from torch.utils.data import random_split
from ray.air import session

def get_imaginette_dataloaders(batch_size):


    transform = v2.Compose([
            v2.RandomRotation(10),
            v2.RandomHorizontalFlip(p=0.5),
            v2.Resize((224,224)),
            v2.ToTensor(),
            v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    dataset = datasets.Imagenette(root="C:/Users/Dominik/Documents/GitHub/DL-models/DLF_PL/data", split="train", download=False, transform=transform, size="full")

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])



    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader



def train_imaginette(config):
    experiment = comet_ml.Experiment(
        api_key=API_KEY,
        project_name="imaginette-tuning",
    )
    print("Starting train")
    
    experiment.log_parameters(config)
    experiment.log_parameter("epochs", config["epochs"])
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    train_loader, val_loader = get_imaginette_dataloaders(config["batch_size"])

    model = ImaginetteCNN()
    model.to(device)
    
    if config["optimizer"] == "adam":
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    else:
        optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)
    
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(config["epochs"]):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        
        # Validation loop
        model.eval()
        correct = 0
        total = 0
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        session.report({"mean_accuracy": accuracy})
    

    experiment.end()

In [ ]:
def custom_trial_name_creator(trial):
    return f"trial_{trial.trial_id}"

def custom_trial_dirname_creator(trial):
    return f"trial_{trial.trial_id}"

config = {
    "lr": tune.loguniform(1e-4, 1e-2),
    "batch_size": tune.choice([16, 32, 64]),
    "optimizer": tune.choice(["adam", "sgd"]),
    "epochs": 5,
}
    
scheduler = ASHAScheduler(
    metric="mean_accuracy",
    mode="max",
    max_t=5,
    grace_period=1,
    reduction_factor=2
)
    
result = tune.run(
    tune.with_parameters(train_imaginette),
    resources_per_trial={"cpu": 2, "gpu": 1 if torch.cuda.is_available() else 0},
    config=config,
    num_samples=5,
    scheduler=scheduler,
    trial_name_creator=custom_trial_name_creator,
    trial_dirname_creator=custom_trial_dirname_creator,
    callbacks=[CometLoggerCallback(api_key=API_KEY,
                                project_name="imaginette-tuning")]
    )
    
best_trial = result.get_best_trial("mean_accuracy", "max", "last") # or 'all' - ask!!
print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation accuracy: {best_trial.last_result['mean_accuracy']}")

2024-12-03 10:34:22,016	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/hazhul/imaginette-tuning/727c2e3802e24d79bcf025fd258036fe

COMET ERROR: Experiment.add_tags: parameter 'tags' must be of type(s) 'list' but None was given
COMET WARNING: Failed to add tag(s) None to the experiment

COMET WARNING: The given value of the metric checkpoint_dir_name was None; ignoring


Trial name,mean_accuracy
trial_c6fa6_00000,18.2154
trial_c6fa6_00001,70.2218
trial_c6fa6_00002,20.3273
trial_c6fa6_00003,72.3865
trial_c6fa6_00004,15.9451


COMET WARNING: The given value of the metric checkpoint_dir_name was None; ignoring
COMET WARNING: The given value of the metric checkpoint_dir_name was None; ignoring
COMET WARNING: The given value of the metric checkpoint_dir_name was None; ignoring
COMET WARNING: The given value of the metric checkpoint_dir_name was None; ignoring
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : trial_c6fa6_00000
COMET INFO:     url                   : https://www.comet.com/hazhul/imaginette-tuning/727c2e3802e24d79bcf025fd258036fe
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     iterations_since_restore [5] : (1, 5)
COMET INFO:     mean_accuracy [5]            : (11.562829989440338, 18.215417106652588)


Best trial config: {'lr': 0.00011793360955543408, 'batch_size': 16, 'optimizer': 'adam', 'epochs': 5}
Best trial final validation accuracy: 72.38648363252376
